In [ ]:

import json
import cv2
import os
from ultralytics import YOLO
from utils import parse_coco_dataset, crop_box, draw_objects, calculate_iou
from YOLOv8_predict import extract_detections_pt


In [ ]:
# initialize paths
folder_with_images = '/data/datasets/model_validation/val_yolo/images'
coco_annotations_file = '/data/datasets/model_validation/coco/annotations/instances_default.json'
visualized_images = 'visualized_images'
yolo_model_name = "yolov8n"

In [ ]:
#  functions
def draw_and_save_coco(coco_annotations, result_folder):
    os.makedirs(result_folder, exist_ok=True)
    for image_filepath, objects in coco_annotations.items():
        image = cv2.imread(image_filepath)
        if image is None:
            print(f"Cannot read image. Check path {image_filepath}")
            continue
        image = draw_objects(image, objects)
        result_filepath = os.path.join(result_folder, os.path.split(image_filepath)[-1])
        cv2.imwrite(result_filepath, image)

def run_image_with_yolo(image, model):
    detections = []
    predicted_data = model(image)
    for pred in predicted_data:
        dets = extract_detections_pt(pred)
        for det in dets:
            label = det['label']
            score = det['score']
            box = crop_box(det['ltrb'], image.shape[:2])
            detections.append({'label': label, 'ltrb': box, 'score': score})
    return detections

def process_images(image_filepaths, model, annotations={}, result_folder=None):
    if result_folder: os.makedirs(result_folder, exist_ok=True)
    images_annotations_detections = {}
    for image_filepath in image_filepaths:
        # read image
        image = cv2.imread(image_filepath)
        if image is None:
            print(f"Cannot read image. Check path {image_filepath}")
            continue
        # get annotated objects if annotations provided
        annotated_objects = annotations.get(image_filepath, [])
        # get detections
        detections = run_image_with_yolo(image, model)
        # draw objects and save an image if required
        if result_folder:
            result_filepath = os.path.join(result_folder, os.path.split(image_filepath)[-1])
            image = draw_objects(image, annotated_objects)
            image = draw_objects(image, detections)
            cv2.imwrite(result_filepath, image)
        #  keep results for next evaluation
        images_annotations_detections[image_filepath] = {
            "annotated_objects": annotated_objects,
            "detections": detections
        }
    return images_annotations_detections

In [ ]:
coco_annotations = parse_coco_dataset(folder_with_images, coco_annotations_file)
#draw_and_save_coco(coco_annotations, visualized_images)

model = YOLO(yolo_model_name)
images_annotations_detections = process_images(coco_annotations.keys(), model, coco_annotations, visualized_images)



In [ ]:


def compute_ious(img_filepath, ground_truth_data, predicted_data):
    results = []
    for gt_obj in ground_truth_data:
        gt_label, gt_box = gt_obj['label'], gt_obj['ltrb']
        for detection in predicted_data:
            det_label, det_box, det_score = detection['label'], detection['ltrb'], detection['score']
            if gt_label == det_label:
                iou = calculate_iou(gt_box, det_box)
                results.append({
                    'img_filepath': img_filepath, 
                    'label': gt_label, 
                    'gt_box': gt_box,
                    'det_box': det_box,
                    'iou': iou})
    return results

img_filepaths = list(coco_annotations.keys())
img_filepath  = img_filepaths[0]
results = compute_ious(img_filepath, images_annotations_detections[img_filepath], images_annotations_detections[img_filepath])
print(results)